In [ ]:
!pip install datasets

import math
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from datasets import load_dataset
from sklearn.metrics import accuracy_score, recall_score
from torch.utils.data import DataLoader

# Configurazione del modello e parametri
#MODEL_NAME = "./model/roberta-base-latin-v2"  # Sostituisci con il tuo modello
MODEL_NAME = "ClassCat/roberta-base-latin-v2"
#DATASET_PATH = "dataset.txt"      # Percorso del tuo dataset
MASK_PROBABILITY = 0.2
BATCH_SIZE = 8

# Caricamento del tokenizer e del modello
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.cuda.set_per_process_memory_fraction(0.95)
torch.cuda.empty_cache()
model.to(device)
model.eval()

# Caricamento del dataset personalizzato
#dataset = load_dataset("parquet", data_dir="./parquet", trust_remote_code=True)
dataset = load_dataset("Cicciokr/CC-100-Latin", revision="refs/convert/parquet")
dataset = dataset['train'].train_test_split(test_size=0.03, shuffle=True)

# Tokenizzazione del dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Tokenizza i dati
tokenized_datasets = dataset['test'].map(tokenize_function, batched=True)

# Creazione del data collator per MLM con mascheramento dinamico
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=MASK_PROBABILITY
)

# Funzione per calcolare Accuratezza e Perplessità
def evaluate_model(dataloader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    all_predictions = []  # Lista per accumulare le predizioni
    all_labels = []  # Lista per accumulare i valori reali

    for batch in dataloader:
        # Prepara un batch di esempio
        #batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
        examples = [tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt") for text in batch["text"]]
        examples_list = batch["text"][:]

        # Combina i batch in un unico dizionario
        input_ids = torch.cat([ex["input_ids"] for ex in examples], dim=0).to(device)
        attention_mask = torch.cat([ex["attention_mask"] for ex in examples], dim=0).to(device)
        labels = input_ids.clone()


        # Applica il data collator per mascherare dinamicamente
        #Sposto i tensori sulla cpu perchè altrimenti il DataCollatorForWholeWordMask non funziona
        batch = data_collator([{
            "input_ids": input_ids[i].cpu().tolist(),
            "attention_mask": attention_mask[i].cpu().tolist(),
            "labels": labels[i].cpu().tolist()
        } for i in range(input_ids.size(0))])

        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)


        with torch.no_grad():
            outputs = model(input_ids, labels=labels)
            logits = outputs.logits
            loss = outputs.loss

        # Calcola il loss totale per la perplessità
        #Valore basso (es. 0.1 - 0.5): Il modello sta facendo previsioni accurate.
        #Valore medio (es. 1 - 2): Prestazioni accettabili, ma con margine di miglioramento.
        #Valore alto (> 2): Modello con scarsa performance; potrebbe richiedere miglioramenti nei dati o nell'addestramento.
        if not torch.isnan(loss):
            total_loss += loss.item() * input_ids.size(0)

            # Calcola l'accuratezza
            predictions = torch.argmax(logits, dim=-1)

            mask = labels != -100  # Maschera per selezionare solo i token mascherati

            correct = (predictions[mask] == labels[mask]).sum().item()
            total_correct += correct
            total_tokens += mask.sum().item()
            # Accumula predizioni e labels
            all_predictions.extend(predictions[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())

    # Accuratezza
    accuracy = total_correct / total_tokens

    # Perplessità
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    recall = recall_score(all_labels, all_predictions, average='weighted')

    return accuracy, perplexity, total_correct, total_loss, total_tokens, recall

# Esegui la valutazione
eval_dataloader = DataLoader(tokenized_datasets, batch_size=BATCH_SIZE, shuffle=False)
accuracy, perplexity, total_correct, total_loss, total_tokens, recall = evaluate_model(eval_dataloader)

# Stampa i risultati
# > 80% Ottimo modello
# = 70 Buon modello
# < 70% Mediocre
print(f"Accuratezza: {accuracy*100:.2f}")
#Se la loss è 0, la perplessità sarà 1, indicando previsioni perfette.
#Perplessità > 1 indica un grado di incertezza crescente
print(f"Perplessità: {perplexity:.4f}")
print(f"Recall: {recall:.4f}")  # Stampa il valore del Recall
print(f"Total correct: {total_correct:.2f}")
print(f"Total Loss: {total_loss:.2f}")
print(f"Total Token: {total_tokens:.2f}")
print(f"Length Dataset: {len(tokenized_datasets):.2f}")

Map:   0%|          | 0/543162 [00:00<?, ? examples/s]